对函数 y=2x ⊤x关于列向量 x求导。

In [1]:
import torch
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

存储梯度

In [2]:
x.requires_grad_(True) #等价于 x = torch.arange(4.0, requires_grad=True)
x.grad #默认是None

计算y

In [3]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

通过调用反向传播函数来自动计算y关于x每个分量的梯度

In [4]:
y.backward() #自动求导
x.grad

tensor([ 0.,  4.,  8., 12.])

In [5]:
x.grad == 4 * x

tensor([True, True, True, True])

计算x的另一个函数

In [ ]:
# 在默认情况下，PyTorch会累积梯度，所以要清除之前的值
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

每个样本独立的偏导数之和

In [6]:
# 对非标量变量进行反向传播时，需要传入一个与该变量同形的梯度参数
x.grad.zero_()
y = x * x
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

将某些计算移动到记录的计算图之外

In [7]:
x.grad.zero_()
y = x * x
u = y.detach()  # 中断y对x的依赖关系
z = u * x
z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [8]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

In [9]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c
a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()
a.grad == d / a,a


(tensor(True), tensor(0.5445, requires_grad=True))